In [17]:
!pip uninstall open3d

^C


In [18]:
!conda install -c open3d-admin open3d


^C


In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from glob import glob
import open3d as o3d
print("Open3D Version: " + o3d.__version__)
# o3d.visualization.webrtc_server.enable_webrtc()
kitti_velodyne_path= r"D:\introspectionBase\datasets\Kitti\raw\training\velodyne"


Open3D Version: 0.11.2


In [4]:
from open3d.j_visualizer import JVisualizer
def load_velodyne_points(filename):
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    return points

def visualize(points):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 3], cmap='viridis')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

if __name__ == '__main__':
    filename = os.path.join(kitti_velodyne_path, '000000.bin')
    points = load_velodyne_points(filename)
    pcd = o3d.geometry.PointCloud()
    
    # Set the points
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    
    # Set the intensity as colors
    pcd.colors = o3d.utility.Vector3dVector(points[:, 3:4].repeat(3, axis=1) / 255)
    # Visualize the point cloud
    visuzlizer = JVisualizer()
    visuzlizer.add_geometry(pcd)
    visuzlizer.show()


JVisualizer with 1 geometries

In [21]:
import open3d as o3d
import numpy as np
def get_yaw_from_rotation_matrix(R):
    """
    Extract the yaw angle from a 3x3 rotation matrix.
    
    Parameters:
        R (numpy.ndarray): 3x3 rotation matrix.
        
    Returns:
        float: Yaw angle in radians.
    """
    # Convert rotation matrix to Euler angles
    yaw_angle = np.arctan2(R[1, 0], R[0, 0])
    
    return yaw_angle
def project_to_axis(points, axis):
    return np.dot(points, axis)

def overlap_on_axis(points1, points2, axis):
    min1 = np.min(project_to_axis(points1, axis))
    max1 = np.max(project_to_axis(points1, axis))
    min2 = np.min(project_to_axis(points2, axis))
    max2 = np.max(project_to_axis(points2, axis))
    overlap = max(0, min(max1, max2) - max(min1, min2))
    return overlap
def create_oriented_box(center, dimensions, yaw):
    R = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    return o3d.geometry.OrientedBoundingBox(center=center, R=R, extent=dimensions)


def rotate_points(points, yaw):
    R = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    return np.dot(points, R.T)

def calculate_iou(box1,box2):
    # Create corner points for both boxes
    center1 = np.array(box1.center)
    center2 = np.array(box2.center)
    dimensions1 = np.array(box1.extent)
    dimensions2 = np.array(box2.extent)
    yaw1 = get_yaw_from_rotation_matrix(box1.R)
    yaw2 = get_yaw_from_rotation_matrix(box2.R)
    half_dims1 = dimensions1 / 2
    half_dims2 = dimensions2 / 2
    corners1 = np.array([np.array([x, y, z]) for x in [-half_dims1[0], half_dims1[0]] for y in [-half_dims1[1], half_dims1[1]] for z in [-half_dims1[2], half_dims1[2]]])
    corners2 = np.array([np.array([x, y, z]) for x in [-half_dims2[0], half_dims2[0]] for y in [-half_dims2[1], half_dims2[1]] for z in [-half_dims2[2], half_dims2[2]]])
    
    # Rotate and translate corners
    corners1 = rotate_points(corners1, yaw1) + center1
    corners2 = rotate_points(corners2, yaw2) + center2
    
    # Calculate axis-aligned bounding boxes for intersection
    min_bound1 = np.min(corners1, axis=0)
    max_bound1 = np.max(corners1, axis=0)
    min_bound2 = np.min(corners2, axis=0)
    max_bound2 = np.max(corners2, axis=0)
    
    # Calculate intersection
    min_intersect = np.maximum(min_bound1, min_bound2)
    max_intersect = np.minimum(max_bound1, max_bound2)
    intersection_dims = np.maximum(0, max_intersect - min_intersect)
    intersection_volume = np.prod(intersection_dims)
    
    # Calculate volumes of the individual boxes
    vol1 = np.prod(dimensions1)
    vol2 = np.prod(dimensions2)
    
    # Calculate IoU
    iou = intersection_volume / (vol1 + vol2 - intersection_volume)
    return iou
# Generate random center for the first box
# center1 = np.random.rand(3) * 10

# # Generate random extents (dimensions) for the first box
# extent1 = np.random.rand(3) * 5

# # Generate random rotation matrix for the first box
# angle1 = np.random.rand() * np.pi
# R1 = o3d.geometry.get_rotation_matrix_from_xyz([0, angle1, 0])

# # Create the first oriented bounding box
# box1 = o3d.geometry.OrientedBoundingBox(center=center1, R=R1, extent=extent1)
# yaw_angle = get_yaw_from_rotation_matrix(box1.R)

# # Generate random translation to ensure some overlap
# translation = np.random.rand(3) * 2 - 1  # Values between -1 and 1

# # Generate random extents (dimensions) for the second box
# extent2 = np.random.rand(3) * 5

# # Generate random rotation matrix for the second box
# angle2 = np.random.rand() * np.pi
# R2 = o3d.geometry.get_rotation_matrix_from_xyz([0, angle2, 0])

# # Create the second oriented bounding box with some overlap
# center2 = center1 + translation
# box2 = o3d.geometry.OrientedBoundingBox(center=center2, R=R2, extent=extent2)
center1 = np.array([0, 0, 0])
dimensions1 = np.array([4, 4, 4])
yaw1 = 0

# Define the properties of the second box (Box B)
# We'll position this box such that it overlaps with Box A to achieve an IoU of ~0.75
center2 = np.array([2, 2, 0])
dimensions2 = np.array([4, 4, 4])
yaw2 = 0

box1 = create_oriented_box(center1, dimensions1, yaw1)
box2 = create_oriented_box(center2, dimensions2, yaw2)
print(calculate_iou(box1, box2))



0.14285714285714285


In [4]:
import random
import os
import numpy as np
kitti_velodyne_path= r"D:\introspectionBase\datasets\Kitti\raw\training\velodyne"

def read_calib_file(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        calibration = {}

        for line in lines:
            # print(line)
            try:
                key, value = line.split(':')
            except ValueError:
                continue
            calibration[key] = np.array([float(x) for x in value.strip().split()])
        return calibration
config_file = r'D:\mmdetection3d\configs\pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint = r'D:\mmdetection3d/ckpts/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth'
test_num  = random.randint(0,7480)
filename = os.path.join(kitti_velodyne_path, f'{test_num:06}.bin')
calib_data = read_calib_file(filename.replace('.bin', '.txt').replace("velodyne","calib"))

In [5]:
calib_data

{'P0': array([721.5377,   0.    , 609.5593,   0.    ,   0.    , 721.5377,
        172.854 ,   0.    ,   0.    ,   0.    ,   1.    ,   0.    ]),
 'P1': array([ 721.5377,    0.    ,  609.5593, -387.5744,    0.    ,  721.5377,
         172.854 ,    0.    ,    0.    ,    0.    ,    1.    ,    0.    ]),
 'P2': array([7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01,
        0.000000e+00, 7.215377e+02, 1.728540e+02, 2.163791e-01,
        0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03]),
 'P3': array([ 7.215377e+02,  0.000000e+00,  6.095593e+02, -3.395242e+02,
         0.000000e+00,  7.215377e+02,  1.728540e+02,  2.199936e+00,
         0.000000e+00,  0.000000e+00,  1.000000e+00,  2.729905e-03]),
 'R0_rect': array([ 0.9999239 ,  0.00983776, -0.00744505, -0.0098698 ,  0.9999421 ,
        -0.00427846,  0.00740253,  0.00435161,  0.9999631 ]),
 'Tr_velo_to_cam': array([ 7.533745e-03, -9.999714e-01, -6.166020e-04, -4.069766e-03,
         1.480249e-02,  7.280733e-04, -9.998902e-01, -7.